In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import pandas as pd
# from google.colab import files

# # 파일 선택 창이 뜹니다. 파일 선택 후 업로드!
# uploaded = files.upload()

# # 업로드된 파일 이름 확인
# for fn in uploaded.keys():
#     print(f'업로드 완료: {fn}')

In [30]:
%cd /content/drive/MyDrive/dev/projects/project1/pill-detect-ai

/content/drive/MyDrive/dev/projects/project1/pill-detect-ai


In [31]:
# 파일 경로만 수정!
csv_path = 'submission_yolo11x_run3_32_50_aug.csv'
mapping_path = 'data/yolo/pill_yolo_format/yolo_to_categoryid.json'

df = pd.read_csv(csv_path)


In [32]:
# 1. 상위 10개
print(df.head(10))

# 2. 컬럼명/타입
print(df.dtypes)

# 3. image_id, category_id 분포
print("image_id 개수:", df['image_id'].nunique())
print("category_id 개수:", df['category_id'].nunique())
print("category_id 샘플:", df['category_id'].unique()[:10])

# 4. annotation_id 중복/결측
print("annotation_id 중복 수:", df['annotation_id'].duplicated().sum())
print("annotation_id 결측 수:", df['annotation_id'].isnull().sum())

# 5. bbox 범위
print(df[['bbox_x', 'bbox_y', 'bbox_w', 'bbox_h']].describe())

# 6. score 분포
print(df['score'].describe())

   annotation_id  image_id  category_id  bbox_x  bbox_y  bbox_w  bbox_h  \
0              1         1        16550     554      63     398     417   
1              2         1        27925     594     672     261     491   
2              3         1         1899     154     249     208     129   
3              4         1        24849     172     741     180     295   
4              5        10        21770     640     286     193     187   
5              6        10        16547      99     804     245     246   
6              7        10        29450     104     226     407     236   
7              8        10         1899     639     840     192     195   
8              9       100        16547     582     121     237     240   
9             10       100         1899     133     238     193     170   

    score  
0  0.8261  
1  0.7891  
2  0.7483  
3  0.5371  
4  0.8390  
5  0.7920  
6  0.7722  
7  0.6856  
8  0.8108  
9  0.7774  
annotation_id      int64
image_id         

In [33]:
import pandas as pd
import json

# 파일 경로만 바꿔줘!

with open(mapping_path, "r", encoding="utf-8") as f:
    valid_category_ids = set([int(v) for v in json.load(f).values()])

# (1) category_id 값이 딕셔너리 값과 정확히 일치하는지 체크!
invalid_category = df[~df['category_id'].isin(valid_category_ids)]
if not invalid_category.empty:
    print(f"❌ [실패] 잘못된 category_id(매핑오류)가 {len(invalid_category)}개 존재!")
    print(invalid_category[['category_id']].drop_duplicates())
else:
    print("✅ [통과] 모든 category_id가 매핑 dict 값과 일치")

# (2) 나머지 포맷도 체크
print("--- 샘플 5개 ---")
print(df.head())

# (3) 유효값/통계(선택)
print("--- category_id 분포 (상위 10개) ---")
print(df['category_id'].value_counts().head(10))


✅ [통과] 모든 category_id가 매핑 dict 값과 일치
--- 샘플 5개 ---
   annotation_id  image_id  category_id  bbox_x  bbox_y  bbox_w  bbox_h  \
0              1         1        16550     554      63     398     417   
1              2         1        27925     594     672     261     491   
2              3         1         1899     154     249     208     129   
3              4         1        24849     172     741     180     295   
4              5        10        21770     640     286     193     187   

    score  
0  0.8261  
1  0.7891  
2  0.7483  
3  0.5371  
4  0.8390  
--- category_id 분포 (상위 10개) ---
category_id
3482     369
3350     140
2482     137
1899     122
16550    107
16547     90
16231     86
36636     80
29666     79
16261     79
Name: count, dtype: int64


In [34]:

# 파일명만 바꿔서 실행!

report = []

# 1. 컬럼명 체크
required_cols = ['annotation_id', 'image_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    report.append(f"❌ [실패] 필수 컬럼 누락: {missing_cols}")
else:
    report.append("✅ [통과] 모든 필수 컬럼이 존재합니다.")

# 2. annotation_id: 중복/결측 체크
ann_dup = df['annotation_id'].duplicated().sum()
ann_null = df['annotation_id'].isnull().sum()
if ann_dup > 0:
    report.append(f"❌ [실패] annotation_id 중복 {ann_dup}개")
else:
    report.append("✅ [통과] annotation_id 중복 없음")
if ann_null > 0:
    report.append(f"❌ [실패] annotation_id 결측 {ann_null}개")
else:
    report.append("✅ [통과] annotation_id 결측 없음")

# 3. image_id: 전부 int이고, 1이상 양수
if not pd.api.types.is_integer_dtype(df['image_id']):
    report.append("❌ [실패] image_id가 int가 아님")
elif (df['image_id'] <= 0).any():
    report.append("❌ [실패] image_id에 0 이하 값이 있음")
else:
    report.append("✅ [통과] image_id 형식 정상")

# 4. category_id: int, 1이상, 값 분포 출력
if not pd.api.types.is_integer_dtype(df['category_id']):
    report.append("❌ [실패] category_id가 int가 아님")
elif (df['category_id'] <= 0).any():
    report.append("❌ [실패] category_id에 0 이하 값이 있음")
else:
    cats = df['category_id'].value_counts()
    report.append(f"✅ [통과] category_id 형식 정상 (샘플: {cats[:5].to_dict()})")

# 5. bbox 좌표/크기: 0 이상, bbox_w/h=0은 경고
bbox_cols = ['bbox_x', 'bbox_y', 'bbox_w', 'bbox_h']
for c in bbox_cols:
    if (df[c] < 0).any():
        report.append(f"❌ [실패] {c}에 음수값 있음")
    else:
        report.append(f"✅ [통과] {c} 0 이상")

if (df['bbox_w'] == 0).any() or (df['bbox_h'] == 0).any():
    report.append("⚠️ [경고] bbox_w 또는 bbox_h에 0이 있음 (채점 무효 가능)")

# 6. score: 0~1 사이, 결측
if df['score'].isnull().any():
    report.append("❌ [실패] score에 결측값 존재")
elif (df['score'] < 0).any() or (df['score'] > 1).any():
    report.append("❌ [실패] score가 0~1 범위를 벗어남")
else:
    report.append("✅ [통과] score 0~1 정상")

# 7. 샘플 3개 표시
report.append("------ 샘플 3개 ------")
report.append(df.head(3).to_string())

# 8. 요약 출력
print("\n".join(report))


✅ [통과] 모든 필수 컬럼이 존재합니다.
✅ [통과] annotation_id 중복 없음
✅ [통과] annotation_id 결측 없음
✅ [통과] image_id 형식 정상
✅ [통과] category_id 형식 정상 (샘플: {3482: 369, 3350: 140, 2482: 137, 1899: 122, 16550: 107})
✅ [통과] bbox_x 0 이상
✅ [통과] bbox_y 0 이상
✅ [통과] bbox_w 0 이상
✅ [통과] bbox_h 0 이상
✅ [통과] score 0~1 정상
------ 샘플 3개 ------
   annotation_id  image_id  category_id  bbox_x  bbox_y  bbox_w  bbox_h   score
0              1         1        16550     554      63     398     417  0.8261
1              2         1        27925     594     672     261     491  0.7891
2              3         1         1899     154     249     208     129  0.7483
